In [9]:
import numpy as np
from QMCP.functions import vmc
import matplotlib.pyplot as plt


def function(alpha, r1, r2, r):
    return(np.exp(-2*r1 - 2*r2 + r/(2*(1+alpha*r))))

def E_loc(alpha, r1_unit, r2_unit, r1_r2, r):
    return(-4 + np.dot((r1_unit - r2_unit), r1_r2) * 1/(r*(1+alpha*r)**2) - 1/(r*(1+alpha*r)**3) - 1/(4*(1+alpha*r)**4) + 1/r)

def six_d_metropolis(N, n_walkers):
    
    x = np.zeros((2,N,n_walkers))
    xyz0 = np.zeros((dim,n_walkers,N))
    xyz1 = np.zeros((dim,n_walkers,N))
    xyz0[:,:,0] = np.random.rand(dim,n_walkers)
    xyz1[:,:,0] = np.random.rand(dim,n_walkers)
    x[0,0,:] = np.linalg.norm(xyz0[:,:,0], axis = 0)
    x[0,0,:] = np.linalg.norm(xyz1[:,:,0], axis = 0)
    trial_displacement = np.zeros((2,dim,n_walkers))
    
    for i in range(N):
        xyz_trial_0 = xyz0[:,:,i] + (0.2*(np.random.rand(dim,n_walkers)*4-2))
        xyz_trial_1 = xyz1[:,:,i] + (0.2*(np.random.rand(dim,n_walkers)*4-2))
        r0 = np.linalg.norm(xyz_trial_0, axis=0)
        r1 = np.linalg.norm(xyz_trial_1, axis=0)
        r_12 = np.linalg.norm(xyz_trial_0 - xyz_trial_1, axis = 0)
        ratio = function(alpha, r0, r1, r_12) / function(alpha, x[0,i-1,:], x[1,i-1,:], np.linalg.norm(xyz0[:,:,i]-xyz1[:,:,i], axis = 0))
        eta = np.random.uniform(0,1,size = n_walkers)
        x[0,i,:] = np.where(ratio >= 1, r0, (np.where(eta < ratio, r0, x[0,i-1,:]))) 
        x[1,i,:] = np.where(ratio >= 1, r1, (np.where(eta < ratio, r0, x[0,i-1,:]))) 
        xyz0[:,:,i] = np.where(ratio >= 1, xyz_trial_0,(np.where(eta < ratio, xyz_trial_0, xyz0[:,:,i-1])))
        xyz1[:,:,i] = np.where(ratio >= 1, xyz_trial_1,(np.where(eta < ratio, xyz_trial_1, xyz1[:,:,i-1])))
        
    r_0 = np.reshape(x[0,1000:,:], n_walkers*N - (n_walkers * 1000))
    r_1 = np.reshape(x[1,1000:,:], n_walkers*N - (n_walkers * 1000))
    xyz0 = np.reshape(xyz0[:,:,1000:], (3,n_walkers*N - (n_walkers * 1000)) )
    xyz1 = np.reshape(xyz1[:,:,1000:], (3,n_walkers*N - (n_walkers * 1000)) )
   
    r_0 = np.where(abs(r_0)<0.005, 0.1 , r_0)
    r_1 = np.where(abs(r_1)<0.005, 0.1 , r_1)
    return(r_0, r_1, xyz0, xyz1)

def calc_unit_vecors():
    r0_unit = xyz_0 / np.l

alpha = 1
dim = 3
walker1, walker2, xyz_0, xyz_1 = six_d_metropolis(5000,5)

print(r_0.shape)

NameError: name 'r_0' is not defined

In [8]:
x = np.linalg.norm(xyz_0, axis = 0)
print(x)
print(walker1)

[ 0.33659785  0.46561221  0.37355334 ...,  0.3776667   0.41278809
  0.19667127]
[ 0.33659785  0.37801477  0.49940826 ...,  0.39422081  0.06597958
  0.19667127]


[[ 0.44671271  0.63508895  0.78579858  0.68561608]
 [ 0.77274892  0.27178438  0.45165792  0.62519822]
 [ 0.85600371  0.34215701  0.49808745  0.61579452]]
[ 0.44671271  0.63508895  0.78579858  0.68561608  0.77274892  0.27178438
  0.45165792  0.62519822  0.85600371  0.34215701  0.49808745  0.61579452]
